### Imports

In [1]:
import os
import time
import sys
import math
import string
import xml.etree.ElementTree as ET
import random
from tabulate import tabulate

### Pre-Processing

In [2]:
test_files = ["A6U.xml", "F98.xml", "J18.xml", "B17.xml", "EW1.xml", "FSS.xml", "CLW.xml",
              "KBC.xml", "KD7.xml", "KSN.xml", "KCU.xml", "KE4.xml", "KP2.xml", "KCV.xml",
              "AC2.xml", "BMW.xml", "G0L.xml", "H9C.xml", "GUU.xml", "CCW.xml",
              "A1H.xml", "A1P.xml", "A3K.xml", "A8N.xml", "A9X.xml", "A2D.xml", "A82.xml", "AAR.xml", "AA3.xml", "AJF.xml", "CBD.xml", "K2C.xml", "K3A.xml", "K58.xml", "K4U.xml", "K29.xml", "A8T.xml", "AHH.xml", "CBM.xml", "A7T.xml", "AL0.xml"]

In [3]:
training_corpus = []
test_corpus = []

def extract_corpus():
    test_data = []
    training_data = []

    texts_directory = "download/Texts"

    for genre in os.listdir(texts_directory):
        if not genre.startswith('.'): # ignores mac hidden file
            for file in os.listdir(os.path.join(texts_directory, genre)):
                if file in test_files:
                    test_data.append(ET.parse(os.path.join(texts_directory, genre, file)))
                else:
                    training_data.append(ET.parse(os.path.join(texts_directory, genre, file)))
    
    # Extracting training data:
    for file in training_data:
        for tree in file.getroot():
            for sentence in tree.iter('s'):
                sen = ['<s>']
                for word in sentence:
                    if word.text is not None and word.text not in string.punctuation: 
                        if word.text[-1] == ' ':
                            sen.append(word.text[:-1].lower()) # removes the space found at the end of most words in the british corpus
                        else:
                            sen.append(word.text.lower())
                sen.append('</s>')

                training_corpus.append(sen)
                
    # Extracting test data:
    for file in test_data:
        for tree in file.getroot():
            for sentence in tree.iter('s'):
                sen = ['<s>']
                for word in sentence:
                    if word.text is not None and word.text not in string.punctuation: 
                        if word.text[-1] == ' ':
                            sen.append(word.text[:-1].lower())
                        else:
                            sen.append(word.text.lower())
                sen.append('</s>')

                test_corpus.append(sen)

In [4]:
words_training = []
words_test = []

def calculate_corpus_stats():
    for sentence in training_corpus:
        for word in sentence:
            words_training.append(word)

    for sentence in test_corpus:
        for word in sentence:
            words_test.append(word)

    vocab_training = set(words_training)
    vocab_test = set(words_test)

    words_total = len(words_training) + len(words_test)
    
    #print("Total number of words in the training corpus: " + str(len(words_training)))
    #print("Unique number of words in the training corpus: " + str(len(vocab_training)))
    #print("")
    #print("Total number of words in the test corpus: " + str(len(words_test)))
    #print("Unique number of words in the test corpus: " + str(len(vocab_test)))
    #print("")
    #print("Test corpus %: " + str(len(words_test) / (words_total) * 100))
    #print("Training corpus %: " + str(len(words_training) / (words_total) * 100))

### Calculating Frequency Counts

In [5]:
unigram_counts = {}
bigram_counts = {}
trigram_counts = {}

def frequency_counter(corpus):
    for sentence in corpus:
        for i in range(len(sentence)):
            unigram = sentence[i]

            if unigram in unigram_counts.keys():
                unigram_counts[unigram] += 1
            else:
                unigram_counts[unigram] = 1
            
    for sentence in corpus:
        for i in range(len(sentence) - 1):
            bigram = (sentence[i], sentence[i+1])

            if bigram in bigram_counts.keys():
                bigram_counts[bigram] += 1
            else:
                bigram_counts[bigram] = 1
                
    for sentence in corpus:
        for i in range(len(sentence) - 2):
            trigram = (sentence[i], sentence[i+1], sentence[i+2])

            if trigram in trigram_counts.keys():
                trigram_counts[trigram] += 1
            else:
                trigram_counts[trigram] = 1

### Vanilla Language Model

In [6]:
P_vanilla_unigram = {}
P_vanilla_bigram = {}
P_vanilla_trigram = {}

def calculate_vanilla():
    for unigram in unigram_counts:
        P_vanilla_unigram[unigram] = unigram_counts[unigram] / len(words_training)
        
    for bigram in bigram_counts:
        P_vanilla_bigram[bigram] = bigram_counts[bigram] / unigram_counts[bigram[0]]
    
    for trigram in trigram_counts:
        P_vanilla_trigram[trigram] = trigram_counts[trigram] / bigram_counts[trigram[0:2]]

### Laplace Language Model

In [7]:
P_laplace_unigram = {}
P_laplace_bigram = {}
P_laplace_trigram = {}
l = len(set(words_training))

def calculate_laplace():
    for unigram in unigram_counts:
        P_laplace_unigram[unigram] = (unigram_counts[unigram] + 1) / (len(words_training) + l)

    for bigram in bigram_counts:
        P_laplace_bigram[bigram] = (bigram_counts[bigram] + 1) / (unigram_counts[bigram[0]] + l)

    for trigram in trigram_counts:
        P_laplace_trigram[trigram] = (trigram_counts[trigram] + 1) / (bigram_counts[trigram[0:2]] + l)

### UNK Language Model

In [8]:
P_unk_unigram = {}
P_unk_bigram = {}
P_unk_trigram = {}

def calculate_unk():
    # Creating a list of all unknown words in the corpus (less than 2 frequency):
    unk_list = []

    for unigram in unigram_counts:
        if unigram_counts[unigram] <= 2:
            unk_list.append(unigram)
        
    unk_set = set(unk_list) # converting to set for faster memberhsip checking
        
    # Replacing all words in the training corpus with frequency less than 2 with <UNK> tag:
    for i, sentence in enumerate(training_corpus):
        for j, word in enumerate(sentence):
            if word in unk_set:
                training_corpus[i][j] = "<UNK>"
            
    frequency_counter(training_corpus)
    
    for unigram in unigram_counts:
        P_unk_unigram[unigram] = unigram_counts[unigram] / len(words_training)
        
    for bigram in bigram_counts:
        P_unk_bigram[bigram] = bigram_counts[bigram] / unigram_counts[bigram[0]]
    
    for trigram in trigram_counts:
        P_unk_trigram[trigram] = trigram_counts[trigram] / bigram_counts[trigram[0:2]]

### Sen_Probability

In [9]:
def sen_probability(sentence, language_model, n_gram):
    P_unigram = 1
    P_bigram = 1
    P_trigram = 1
    
    if language_model == "vanilla":
        if n_gram == "unigram":
            for i in range(len(sentence)):
                if sentence[i] in P_vanilla_unigram.keys():
                    P_unigram *= P_vanilla_unigram[sentence[i]]
                else: P_unigram *= 0
            return P_unigram
        elif n_gram == "bigram":
            for i in range(len(sentence) - 1):
                sequence = (sentence[i], sentence[i+1])
                if sequence in P_vanilla_bigram.keys():
                    P_bigram *= P_vanilla_bigram[sequence]
                else: P_bigram *= 0
            return P_bigram
        elif n_gram == "trigram":
            for i in range(len(sentence) - 2):
                sequence = (sentence[i], sentence[i+1], sentence[i+2])
                if sequence in P_vanilla_trigram.keys():
                    P_trigram *= P_vanilla_trigram[sequence]
                else: P_trigram *= 0
            return P_trigram
        else: print("Invalid n-gram input")
            
    elif language_model == "laplace":
        if n_gram == "unigram":
            for i in range(len(sentence)):
                if sentence[i] in P_laplace_unigram.keys():
                    P_unigram *= P_laplace_unigram[sentence[i]]
                else: P_unigram *= 0
            return P_unigram
        elif n_gram == "bigram":
            for i in range(len(sentence) - 1):
                sequence = (sentence[i], sentence[i+1])
                if sequence in P_laplace_bigram.keys():
                    P_bigram *= P_laplace_bigram[sequence]
                else: P_bigram *= 0
            return P_bigram
        elif n_gram == "trigram":
            for i in range(len(sentence) - 2):
                sequence = (sentence[i], sentence[i+1], sentence[i+2])
                if sequence in P_laplace_trigram.keys():
                    P_trigram *= P_laplace_trigram[sequence]
                else: P_trigram *= 0
            return P_trigram
        else: print("Invalid n-gram input")
            
    elif language_model == "unk":
        if n_gram == "unigram":
            for i in range(len(sentence)):
                if sentence[i] in P_unk_unigram.keys():
                    P_unigram *= P_unk_unigram[sentence[i]]
                else: P_unigram *= 0
            return P_unigram
        elif n_gram == "bigram":
            for i in range(len(sentence) - 1):
                sequence = (sentence[i], sentence[i+1])
                if sequence in P_unk_bigram.keys():
                    P_bigram *= P_unk_bigram[sequence]
                else: P_bigram *= 0
            return P_bigram
        elif n_gram == "trigram":
            for i in range(len(sentence) - 2):
                sequence = (sentence[i], sentence[i+1], sentence[i+2])
                if sequence in P_unk_trigram.keys():
                    P_trigram *= P_unk_trigram[sequence]
                else: P_trigram *= 0
            return P_trigram
        else: print("Invalid n-gram input")
            
    else: print("Invalid languge model input")

### Linear Interpolation

In [10]:
def linear_interpolation(language_model, sentence):
    trigram_lambda = 0.6
    bigram_lambda = 0.3
    unigram_lambda = 0.1
    
    sentence_probabilities = {}
    
    if language_model == "vanilla":
        sentence_probabilities["unigram"] = sen_probability(sentence, "vanilla", "unigram")
        sentence_probabilities["bigram"] = sen_probability(sentence, "vanilla", "bigram")
        sentence_probabilities["trigram"] = sen_probability(sentence, "vanilla", "trigram")
        
    elif language_model == "laplace":
        sentence_probabilities["unigram"] = sen_probability(sentence, "laplace", "unigram")
        sentence_probabilities["bigram"] = sen_probability(sentence, "laplace", "bigram")
        sentence_probabilities["trigram"] = sen_probability(sentence, "laplace", "trigram")
        
    elif language_model == "unk":
        sentence_probabilities["unigram"] = sen_probability(sentence, "unk", "unigram")
        sentence_probabilities["bigram"] = sen_probability(sentence, "unk", "bigram")
        sentence_probabilities["trigram"] = sen_probability(sentence, "unk", "trigram")
        
    else: print("Invalid language model Input")

    P_interpolated = (unigram_lambda * sentence_probabilities["unigram"]) + (bigram_lambda * sentence_probabilities["bigram"]) + (trigram_lambda * sentence_probabilities["trigram"]) 

    return P_interpolated

### Perplexity Table

In [11]:
def calculate_perplexity_table(corpus):
    language_models = ["vanilla", "laplace", "unk"]
    ngrams = ["unigram", "bigram", "trigram", "linear-interpolation"]
    pp = {}
    
    for language_model in language_models:
        for ngram in ngrams:
            probabilities = []
            count = 0
            key = (language_model, ngram)
            if ngram != "linear-interpolation":
                for sentence in corpus:
                    p = sen_probability(sentence, language_model, ngram)
                    if p != 0:
                        probabilities.append(p)
                        count += len(sentence)
            else:
                for sentence in corpus:
                    p = linear_interpolation(language_model, sentence)
                    if p != 0:
                        probabilities.append(p)
                        count += len(sentence)
    
            pp[key] = pow(2, -sum(math.log2(p) for p in probabilities) / count)
        
    data = [["       ", "Unigram",                   "Bigram",                    "Trigram",                 "Linear Interpolation"],
            ["Vanilla", pp[("vanilla", "unigram")],  pp[("vanilla", "bigram")],   pp["vanilla", "trigram"],  pp["vanilla", "linear-interpolation"]],
            ["Laplace", pp[("laplace", "unigram")],  pp["laplace", "bigram"],     pp["laplace", "trigram"],  pp["laplace", "linear-interpolation"]],
            ["UNK",     pp["unk", "unigram"],        pp[("unk", "bigram")],       pp["unk", "trigram"],      pp["unk", "linear-interpolation"]]]
    
    print(tabulate(data))

### Generate Sentence

In [12]:
def generate_sentence():
    while True:
        language_model = input("Select a Language Model:\nVanilla\nLaplace\nUNK\n")
        language_model = language_model.lower()
            
        if language_model == "vanilla" or language_model == "laplace" or language_model == "unk":
            break
        else:
            print("Invalid Input")
    
    print("Input a phrase")
    phrase = input()
    print("")
    
    new_sentence = phrase.split()
    new_sentence.insert(0, "<s>")
    
    new_sentence = [x.lower() for x in new_sentence]
    
    next_word = ""
    
    print(phrase, end = ' ')
    
    while next_word != "</s>":
        possible_words = []
        probabilities = []
        
        context = tuple(new_sentence[-2:])

        if language_model == "vanilla":
            for key in P_vanilla_trigram.keys():
                if key[0:2] == context:
                    possible_words.append(key)
                    probabilities.append(P_vanilla_trigram[key])
        elif language_model == "laplace":
            for key in P_laplace_trigram.keys():
                if key[0:2] == context:
                    possible_words.append(key)
                    probabilities.append(P_laplace_trigram[key])
        elif language_model == "unk":
             for key in P_unk_trigram.keys():
                if key[0:2] == context:
                    possible_words.append(key)
                    probabilities.append(P_unk_trigram[key])
                    
        choice = random.choices(possible_words, weights = probabilities, k=1)

        next_word = choice[0][2]
        
        new_sentence.append(next_word)
        
        if next_word == '</s>':
            print(".", end = ' ')
        elif next_word == 'i':
            print(next_word.capitalize(), end = ' ')
        else: print(next_word, end = ' ')

In [13]:
start = time.time()

extract_corpus()
calculate_corpus_stats()
frequency_counter(training_corpus)
calculate_vanilla()
calculate_laplace()
calculate_unk()

end = time.time()
print(str(end-start) + " seconds")

28.957848072052002 seconds


In [14]:
calculate_perplexity_table(test_corpus)

-------  -----------------  ------------------  ------------------  --------------------
         Unigram            Bigram              Trigram             Linear Interpolation
Vanilla  519.5210327818343  30.623874930158117  3.911211062659215   305.1950335793441
Laplace  513.5022334049727  29.4037458259578    3.6844946467983966  296.6078563146629
UNK      261.6247518922054  30.62828139570548   3.9116638189738113  186.2329985227002
-------  -----------------  ------------------  ------------------  --------------------


In [15]:
generate_sentence()

Select a Language Model:
Vanilla
Laplace
UNK
vanilla
Input a phrase
every time

every time he had an audience shuffling , coughing in the public ’ and to the loose ball and conceded a penalty decision . 

In [17]:
memory_vanilla = sys.getsizeof(P_vanilla_unigram) + sys.getsizeof(P_vanilla_bigram) + sys.getsizeof(P_vanilla_trigram)
memory_laplace = sys.getsizeof(P_laplace_unigram) + sys.getsizeof(P_laplace_bigram) + sys.getsizeof(P_laplace_trigram)
memory_unk = sys.getsizeof(P_unk_unigram) + sys.getsizeof(P_unk_bigram) + sys.getsizeof(P_unk_trigram)

print("Memory required for Vanilla Language Model Dictionary: " + str(memory_vanilla) + " bytes")
print("Memory required for Laplace Language Model Dictionary: " + str(memory_laplace) + " bytes")
print("Memory required for UNK Language Model Dictionary: " + str(memory_unk) + " bytes")

Memory required for Vanilla Language Model Dictionary: 128450840 bytes
Memory required for Laplace Language Model Dictionary: 128450840 bytes
Memory required for UNK Language Model Dictionary: 128450840 bytes


In [18]:
sentence = "It was a nice day"

sentence = sentence.split()
sentence = [x.lower() for x in sentence]
sentence.insert(0, "<s>")
sentence.append("</s>")
sentence

['<s>', 'it', 'was', 'a', 'nice', 'day', '</s>']

In [19]:
print("Vanilla: " + str(sen_probability(sentence, "vanilla", "bigram")))
print("Laplace: " + str(sen_probability(sentence, "laplace", "bigram")))
print("UNK: " + str(sen_probability(sentence, "unk", "bigram")))

Vanilla: 1.5871509696627568e-09
Laplace: 1.7437044228039855e-09
UNK: 1.5871509696627568e-09
